In [1]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import os
import pandas as pd
import numpy as np
# import seaborn as sns
from utilities import* 
from pandas import read_excel
from matplotlib import pyplot as plt
plt.rc('pdf',fonttype = 42)
plt.rc('ps',fonttype = 42)
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage,AnchoredOffsetbox
from matplotlib import rc, rcParams
rc('font', weight='bold')
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pymongo
from pymongo import MongoClient
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
from os import listdir
from os.path import isfile, join
import utilities as ut
ut.styles['WDG'] = '-.'
ut.styles['WDG+BP'] =(0, (1, 1))
ut.styles['WDG+CH'] = (0, (5, 1))
ut.styles['WDG+CHBP'] = '-'

ut.colors['WDG'] = 'C0'
ut.colors['WDG+BP'] = 'C1'
ut.colors['WDG+CH'] = 'C2'
ut.colors['WDG+CHBP'] = 'C3'
import datetime
from pprint import pprint

In [28]:
def create_algo_detail(db):
    LazyCBS = {
        "algo_name": "Lazy-CBS",
        "authors": "Gange et al.",
        "papers": "Graeme Gange, Daniel Harabor and Peter J. Stuckey. Lazy CBS: Implicit Conflict-Based Search Using Lazy Clause Generation. In Proceedings of the Twenty-Ninth International Conference on Automated Planning and Scheduling 2018.",
        "github": "https://bitbucket.org/gkgange/lazycbs/src/master/",
        "comments": "",
        'user_id':""
    }
    db['algorithms'].insert_one(LazyCBS)
    
def get_path(filename,number_of_agents):
    with open(filename) as file:
        planString = ""
        number_of_line = 0
        for line in file:
            number_of_line = number_of_line + 1
            planString  = planString + line
        planString = planString.rstrip("\n")
        if(number_of_agents != number_of_line):
            print("ohhhhhhhh fuck!!!!!!!!!!!!!!!!!!")
            print("ohhhhhhhh fuck!!!!!!!!!!!!!!!!!!")
            print(number_of_agents)
            print(number_of_line)
            print("Agents number not matched")
    return planString

def insert_submission(db,submission,instance_id,algo_name,path):
    current_instance = db['instances'].find_one({'_id':instance_id})
    if(submission['solution_cost']!= None):
        if(current_instance['lower_cost'] != None):
            if(current_instance['lower_cost'] > submission['solution_cost']):
                print("Inserting error: existing lower cost larger than submission solution cost")
#         //insert solution
        if(current_instance['solution_cost'] == None):
            submission["best_solution"]= True
            path_id = db['solution_paths'].insert_one({"instance_id": instance_id, 'solution_path':path}).inserted_id
            db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                              {"solution_cost": submission['solution_cost'],
                                                                "solution_date":submission['date'],
                                                                "solution_path_id": path_id,
                                                                "empty": False
                                                              },
                                                              "$push":
                                                              {
                                                                 "solution_algos":{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                 }
                                                              }
                                                             })
        else:
            if(current_instance['solution_cost'] > submission['solution_cost']):
                db['submissions'].update_many({"instance_id" :  instance_id},{"$set": 
                                                      { 
                                                          "best_solution" : False
                                                      }
                                                     })
                submission["best_solution"]= True
                path_id = db['solution_paths'].insert_one({"instance_id": instance_id, 'solution_path':path}).inserted_id
                db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                                    {
                                                                "solution_cost": submission['solution_cost'],
                                                                "solution_algos":[{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                    }],
                                                                "solution_date": submission['date'],
                                                                "solution_path_id": path_id,
                                                                "empty": False
                                                                    }
                                                             })
                
            elif (current_instance['solution_cost'] == submission['solution_cost']):
                submission["best_solution"]= True
                solution_inserted = False
                for algo in current_instance["solution_algos"]:
                    if(algo['algo_name'] == algo_name):
                        solution_inserted = True
                        
                if(solution_inserted == False):
                    db['instances'].update_one({"_id" : instance_id},{
                                                      "$push":
                                                      {
                                                         "solution_algos":{
                                                             "algo_name": algo_name,
                                                             "algo_id": submission['algo_id'],
                                                             "date":submission['date'],
                                                         }
                                                      }
                                                     })   
          
    if(submission['lower_cost'] != None):
        if(current_instance['solution_cost'] != None):
            if(submission['lower_cost'] > current_instance['solution_cost']):
                print("Inserting error: submission lower cost larger than existing solution cost")
        if(current_instance['lower_cost'] == None):
            submission["best_lower"]= True
            db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                              {"lower_cost": submission['lower_cost'],
                                                                "lower_date":submission['date'],
                                                                "empty": False
                                                              },
                                                              "$push":
                                                              {
                                                                 "lower_algos":{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                 }
                                                              }
                                                             })
        
        else:
            if(current_instance['lower_cost'] < submission['lower_cost']):
                db['submissions'].update_many({"instance_id" :  instance_id},{"$set": 
                                                      { 
                                                          "best_lower" : False
                                                      }
                                                     })
                submission["best_lower"]= True
                db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                                    {
                                                                "lower_cost": submission['lower_cost'],
                                                                "lower_algos":[{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                    }],
                                                                "lower_date": submission['date'],
                                                                "empty": False
                                                                    }
                                                             })
            elif (current_instance['lower_cost'] == submission['lower_cost']):
                submission["best_lower"]= True
                lower_inserted = False
                for algo in current_instance["lower_algos"]:
                    if(algo['algo_name'] == algo_name):
                        lower_inserted = True
                if(lower_inserted == False):
                    db['instances'].update_one({"_id" : instance_id},{
                                                      "$push":
                                                      {
                                                         "lower_algos":{
                                                             "algo_name": algo_name,
                                                             "algo_id": submission['algo_id'],
                                                             "date":submission['date'],
                                                         }
                                                      }
                                                     })  
        
    current_instance = db['instances'].find_one({'_id':instance_id})
    if current_instance['lower_cost'] == current_instance['solution_cost']:
        db['instances'].update_one({"_id" : instance_id},{"$set": 
                          {"closed": True}
                         })  
#     db['submissions'].insert_one(submission)
    db['submissions'].update_one({"instance_id":submission["instance_id"], "map_id":submission["map_id"],"algo_id":submission["algo_id"]}, 
                                 {"$set":
                                     {"lower_cost":submission['lower_cost'], 
                                      "solution_cost" : submission[ "solution_cost"],
                                      "date" : submission["date"],
                                      "best_lower" : submission["best_lower"],
                                      "best_solution" : submission["best_solution"]
                                     
                                     
                                     }
                                 }, upsert=True)


def verify_submission(db,submission,instance_id,algo_name,path):
    current_instance = db['instances'].find_one({'_id':instance_id})
    if(submission['solution_cost']!= None):
        if(current_instance['lower_cost'] != None):
            if(current_instance['lower_cost'] > submission['solution_cost']):
                print("submission:"+ str(submission['solution_cost']))
                print("current:"+ str(current_instance['lower_cost']))
                print("Inserting error: existing lower cost larger than submission solution cost")
    if(submission['lower_cost'] != None):
        if(current_instance['solution_cost'] != None):
            if(submission['lower_cost'] > current_instance['solution_cost']):
#                 print("")  
                print("submission:"+ str(submission['lower_cost']))
                print("current:"+ str(current_instance['solution_cost']))
                print("Inserting error: submission lower cost larger than existing solution cost")  
            
def valid_results(db,map_name):
    scen_type = ["even","random"]
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    for i in range(1, 26):
        for scen_t in scen_type:
            print("validing: "+ map_name + " "+scen_t + " "+ str(i))
            scen_data =  db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})
            scen_id = scen_data['_id']
            number_agent = scen_data['instances']
            for k in range(1,number_agent):
                instance_id = db['instances'].find_one({"scen_id": scen_id ,"agents": k})['_id']
                current_instance = db['instances'].find_one({'_id':instance_id})
                if(current_instance['lower_cost'] != None and current_instance['solution_cost'] != None):
                    if(current_instance['lower_cost'] > current_instance['solution_cost'] ):
                        print('error: ' + str(i) + " " + scen_t + " "+str(k))
                        print('error: ' + str(current_instance['lower_cost']) +" "+str(current_instance['solution_cost']))
                
                
                
def insert_map_results(db,map_name):
    algo_id = db['algorithms'].find_one({"algo_name":"Lazy-CBS"})['_id']      
    algo_name = "Lazy-CBS"
    scen_type = ["even","random"]
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    for i in range(1, 26):
        for scen_t in scen_type:
            result_file = "../results/"+map_name+"/statistic/"+map_name+"-"+scen_t+"-"+str(i)+"-60"
            if( os.path.exists(result_file) == False):
                continue
            print(result_file)
            df1 = load_data(result_file)
            scen_data =  db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})
            scen_id = scen_data['_id']
            number_agent = scen_data['instances']
            for index, row in df1.iterrows():
                if(row['num_agents'] > number_agent):
                    break;
                instance_id = db['instances'].find_one({"scen_id": scen_id ,"agents": row['num_agents']})['_id']
                submission= {
                        "map_id": map_id,
                        "instance_id" :  instance_id,
                        "algo_id": algo_id,
                        "lower_cost" : None,
                        "solution_cost": None,
                        "date": str(datetime.datetime.now().date()),
                        "best_lower": False,
                        "best_solution": False,
                        "scen_id":scen_id,
                        "agents":row['#agents']
                    }
                path = None
                if row['solution cost'] != -1:
                    path_results =  "../results/"+map_name+"/path/"+map_name+"-"+scen_t+"-"+str(i)+"-60/"+str(row['num_agents']) +"-path.txt"
                    path = get_path(path_results,row['num_agents'])
                    submission["solution_cost"] = row['solution cost']
                if row['lower_bound'] != -1:
                    submission["lower_cost"] = row['lower_bound']
#                 verify_submission(db,submission,instance_id,algo_name,path)
                insert_submission(db,submission,instance_id,algo_name,path)

def update_map_progress(db, map_name):
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    scen_type = ["even","random"]
    total_s = 0
    total_l = 0
    for i in range(1, 26):
        for scen_t in scen_type: 
            scen_id = db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})['_id']
        
            num_s = db["instances"].count_documents({"scen_id" : scen_id, "closed" : True})
            num_l = db["instances"].count_documents({"scen_id" : scen_id, "solution_cost" : {"$ne":None}})
            total_s = total_s + num_s
            total_l = total_l + num_l
            db['scenarios'].update_one({"_id" : scen_id}, {"$set":{
                "instances_closed" : num_s,
                "instances_solved" : num_l
            }})
    db['maps'].update_one({"_id" : map_id}, {"$set":{
                "instances_closed" : total_s,
                "instances_solved" : total_l
            }})
    
    
def upload_map_results(db, map_name):
    insert_map_results(db,map_name)
    update_map_progress(db, map_name)
    
def update_algo_data(db):
    algorithms = db['algorithms'].find({})
    for algo in algorithms: 
        bb = db['submissions'].count_documents({"algo_id": algo['_id'], "best_lower": True})
        cc = db['submissions'].count_documents({"algo_id": algo['_id'], "best_solution": True})
        dd = db['submissions'].count_documents({"algo_id": algo['_id'], "$expr": {"$eq": ["$lower_cost", "$solution_cost"]}})
        db['algorithms'].update_one({"_id" : algo['_id']},{"$set":{ "best_lower":bb, "best_solution":cc, "instances_closed":dd}})   
        
def update_algo_data(db):
    algorithms = db['algorithms'].find({})
    for algo in algorithms: 
        bb = db['submissions'].count_documents({"algo_id": algo['_id'], "best_lower": True})
        cc = db['submissions'].count_documents({"algo_id": algo['_id'], "best_solution": True})
        dd = db['submissions'].count_documents({"algo_id": algo['_id'], "$expr": {"$eq": ["$lower_cost", "$solution_cost"]}})
        ee = db['submissions'].count_documents({"algo_id": algo['_id'], "$expr": {"$ne": [ "$solution_cost", None]}})
        db['algorithms'].update_one({"_id" : algo['_id']},{"$set":{ "best_lower":bb, "best_solution":cc, "instances_closed":dd,"instances_solved":ee}})   
            

In [26]:
url = "mongodb://localhost:27017/"
client = pymongo.MongoClient(url)
db = client['MAPF_V2']
print("Connect successfully")
update_algo_data(db)

Connect successfully


In [29]:
url = "mongodb://localhost:27017/"
client = pymongo.MongoClient(url)
db = client['MAPF_V2']
print("Connect successfully")
finished_map=[]
# create_algo_detail(db)
folder = '../results'
sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
for map_name in sub_folders:
    if map_name in finished_map:
        print("Skip finished map: " + map_name)
    else:
        upload_map_results(db,map_name)

Connect successfully
../results/orz900d/statistic/orz900d-even-4-60
../results/orz900d/statistic/orz900d-even-6-60
../results/orz900d/statistic/orz900d-random-8-60
../results/orz900d/statistic/orz900d-even-9-60
../results/orz900d/statistic/orz900d-even-10-60
../results/orz900d/statistic/orz900d-random-10-60
../results/orz900d/statistic/orz900d-random-11-60
../results/orz900d/statistic/orz900d-random-13-60
../results/orz900d/statistic/orz900d-even-15-60
../results/orz900d/statistic/orz900d-random-15-60
../results/orz900d/statistic/orz900d-random-18-60
../results/orz900d/statistic/orz900d-random-19-60
../results/orz900d/statistic/orz900d-even-22-60
../results/orz900d/statistic/orz900d-random-22-60
../results/orz900d/statistic/orz900d-random-23-60
../results/orz900d/statistic/orz900d-random-24-60
../results/orz900d/statistic/orz900d-random-25-60
passed
../results/maze-32-32-4/statistic/maze-32-32-4-even-1-60
../results/maze-32-32-4/statistic/maze-32-32-4-random-1-60
../results/maze-32-32-

../results/w_woundedcoast/statistic/w_woundedcoast-even-23-60
../results/w_woundedcoast/statistic/w_woundedcoast-random-23-60
../results/w_woundedcoast/statistic/w_woundedcoast-even-24-60
../results/w_woundedcoast/statistic/w_woundedcoast-random-24-60
../results/w_woundedcoast/statistic/w_woundedcoast-even-25-60
../results/w_woundedcoast/statistic/w_woundedcoast-random-25-60
passed
../results/empty-48-48/statistic/empty-48-48-even-1-60
../results/empty-48-48/statistic/empty-48-48-random-1-60
../results/empty-48-48/statistic/empty-48-48-even-2-60
../results/empty-48-48/statistic/empty-48-48-random-2-60
../results/empty-48-48/statistic/empty-48-48-even-3-60
../results/empty-48-48/statistic/empty-48-48-random-3-60
../results/empty-48-48/statistic/empty-48-48-even-4-60
../results/empty-48-48/statistic/empty-48-48-random-4-60
../results/empty-48-48/statistic/empty-48-48-even-5-60
../results/empty-48-48/statistic/empty-48-48-random-5-60
../results/empty-48-48/statistic/empty-48-48-even-6-60


../results/den312d/statistic/den312d-random-20-60
../results/den312d/statistic/den312d-even-21-60
../results/den312d/statistic/den312d-random-21-60
../results/den312d/statistic/den312d-even-22-60
../results/den312d/statistic/den312d-random-22-60
../results/den312d/statistic/den312d-even-23-60
../results/den312d/statistic/den312d-random-23-60
../results/den312d/statistic/den312d-even-24-60
../results/den312d/statistic/den312d-random-24-60
../results/den312d/statistic/den312d-even-25-60
../results/den312d/statistic/den312d-random-25-60
passed
../results/Boston_0_256/statistic/Boston_0_256-even-1-60
../results/Boston_0_256/statistic/Boston_0_256-random-1-60
../results/Boston_0_256/statistic/Boston_0_256-even-2-60
../results/Boston_0_256/statistic/Boston_0_256-random-2-60
../results/Boston_0_256/statistic/Boston_0_256-even-3-60
../results/Boston_0_256/statistic/Boston_0_256-random-3-60
../results/Boston_0_256/statistic/Boston_0_256-even-4-60
../results/Boston_0_256/statistic/Boston_0_256-r

../results/random-32-32-20/statistic/random-32-32-20-random-23-60
../results/random-32-32-20/statistic/random-32-32-20-even-24-60
../results/random-32-32-20/statistic/random-32-32-20-random-24-60
../results/random-32-32-20/statistic/random-32-32-20-even-25-60
../results/random-32-32-20/statistic/random-32-32-20-random-25-60
passed
../results/den520d/statistic/den520d-even-1-60
../results/den520d/statistic/den520d-random-1-60
../results/den520d/statistic/den520d-even-2-60
../results/den520d/statistic/den520d-random-2-60
../results/den520d/statistic/den520d-even-3-60
../results/den520d/statistic/den520d-random-3-60
../results/den520d/statistic/den520d-even-4-60
../results/den520d/statistic/den520d-random-4-60
../results/den520d/statistic/den520d-even-5-60
../results/den520d/statistic/den520d-random-5-60
../results/den520d/statistic/den520d-even-6-60
../results/den520d/statistic/den520d-random-6-60
../results/den520d/statistic/den520d-even-7-60
../results/den520d/statistic/den520d-random-

../results/random-32-32-10/statistic/random-32-32-10-even-24-60
../results/random-32-32-10/statistic/random-32-32-10-random-24-60
../results/random-32-32-10/statistic/random-32-32-10-even-25-60
../results/random-32-32-10/statistic/random-32-32-10-random-25-60
passed
../results/lak303d/statistic/lak303d-even-1-60
../results/lak303d/statistic/lak303d-random-1-60
../results/lak303d/statistic/lak303d-even-2-60
../results/lak303d/statistic/lak303d-random-2-60
../results/lak303d/statistic/lak303d-even-3-60
../results/lak303d/statistic/lak303d-random-3-60
../results/lak303d/statistic/lak303d-even-4-60
../results/lak303d/statistic/lak303d-random-4-60
../results/lak303d/statistic/lak303d-even-5-60
../results/lak303d/statistic/lak303d-random-5-60
../results/lak303d/statistic/lak303d-even-6-60
../results/lak303d/statistic/lak303d-random-6-60
../results/lak303d/statistic/lak303d-even-7-60
../results/lak303d/statistic/lak303d-random-7-60
../results/lak303d/statistic/lak303d-even-8-60
../results/lak

...

../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-even-22-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-random-22-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-even-23-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-random-23-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-even-24-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-random-24-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-even-25-60
../results/warehouse-10-20-10-2-2/statistic/warehouse-10-20-10-2-2-random-25-60
passed
../results/room-32-32-4/statistic/room-32-32-4-even-1-60
../results/room-32-32-4/statistic/room-32-32-4-random-1-60
../results/room-32-32-4/statistic/room-32-32-4-even-2-60
../results/room-32-32-4/statistic/room-32-32-4-random-2-60
../results/room-32-32-4/statistic/room-32-32-4-even-3-60
../results/room-32-32-4/statistic/room-32-32-4-random-3-60
../results/ro

../results/room-64-64-16/statistic/room-64-64-16-even-22-60
../results/room-64-64-16/statistic/room-64-64-16-random-22-60
../results/room-64-64-16/statistic/room-64-64-16-even-23-60
../results/room-64-64-16/statistic/room-64-64-16-random-23-60
../results/room-64-64-16/statistic/room-64-64-16-even-24-60
../results/room-64-64-16/statistic/room-64-64-16-random-24-60
../results/room-64-64-16/statistic/room-64-64-16-even-25-60
../results/room-64-64-16/statistic/room-64-64-16-random-25-60
passed
../results/maze-128-128-10/statistic/maze-128-128-10-even-1-60
../results/maze-128-128-10/statistic/maze-128-128-10-random-1-60
../results/maze-128-128-10/statistic/maze-128-128-10-even-2-60
../results/maze-128-128-10/statistic/maze-128-128-10-random-2-60
../results/maze-128-128-10/statistic/maze-128-128-10-even-3-60
../results/maze-128-128-10/statistic/maze-128-128-10-random-3-60
../results/maze-128-128-10/statistic/maze-128-128-10-even-4-60
../results/maze-128-128-10/statistic/maze-128-128-10-rando

../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-even-12-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-random-12-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-even-13-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-random-13-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-even-14-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-random-14-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-even-15-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-random-15-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-even-16-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-random-16-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-even-17-60
../results/warehouse-20-40-10-2-1/statistic/warehouse-20-40-10-2-1-random-17-60
../results/warehouse-20-40-10-2-1/statistic/warehous

../results/empty-16-16/statistic/empty-16-16-random-17-60
../results/empty-16-16/statistic/empty-16-16-even-18-60
../results/empty-16-16/statistic/empty-16-16-random-18-60
../results/empty-16-16/statistic/empty-16-16-even-19-60
../results/empty-16-16/statistic/empty-16-16-random-19-60
../results/empty-16-16/statistic/empty-16-16-even-20-60
../results/empty-16-16/statistic/empty-16-16-random-20-60
../results/empty-16-16/statistic/empty-16-16-even-21-60
../results/empty-16-16/statistic/empty-16-16-random-21-60
../results/empty-16-16/statistic/empty-16-16-even-22-60
../results/empty-16-16/statistic/empty-16-16-random-22-60
../results/empty-16-16/statistic/empty-16-16-even-23-60
../results/empty-16-16/statistic/empty-16-16-random-23-60
../results/empty-16-16/statistic/empty-16-16-even-24-60
../results/empty-16-16/statistic/empty-16-16-random-24-60
../results/empty-16-16/statistic/empty-16-16-even-25-60
../results/empty-16-16/statistic/empty-16-16-random-25-60
passed
../results/maze-128-128

../results/brc202d/statistic/brc202d-even-18-60
../results/brc202d/statistic/brc202d-random-18-60
../results/brc202d/statistic/brc202d-even-19-60
../results/brc202d/statistic/brc202d-random-19-60
../results/brc202d/statistic/brc202d-even-20-60
../results/brc202d/statistic/brc202d-random-20-60
../results/brc202d/statistic/brc202d-even-21-60
../results/brc202d/statistic/brc202d-random-21-60
../results/brc202d/statistic/brc202d-even-22-60
../results/brc202d/statistic/brc202d-random-22-60
../results/brc202d/statistic/brc202d-even-23-60
../results/brc202d/statistic/brc202d-random-23-60
../results/brc202d/statistic/brc202d-even-24-60
../results/brc202d/statistic/brc202d-random-24-60
../results/brc202d/statistic/brc202d-even-25-60
../results/brc202d/statistic/brc202d-random-25-60
passed
../results/warehouse-10-20-10-2-1/statistic/warehouse-10-20-10-2-1-even-1-60
../results/warehouse-10-20-10-2-1/statistic/warehouse-10-20-10-2-1-random-1-60
../results/warehouse-10-20-10-2-1/statistic/warehouse

In [19]:
def rewrite_hearder(filename):
    open(filename+"_re", "w").write("runtime,solution cost,lower bound,upper bound,instance name,#agents\n" + "".join(open(filename).readlines()[1:]))

In [20]:
folder = '../results'
sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
for map_name in sub_folders:
    scen_type = ["even","random"]
    for i in range(1, 26):
        for scen_t in scen_type:
            filename = "../results/"+map_name+"/statistic/"+map_name+"-"+scen_t+"-"+str(i)+"-60"
            rewrite_hearder(filename)
